In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, roc_curve, auc
from custom_functions import processing
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFECV
import statistics

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
file_path = 'P:\DATA_OCT_22\Expert_Eye\Dataset\Data\gait_posture.xlsx'
dataset = pd.read_excel(file_path)
#dataset = dataset.drop(['Foldername'], axis=1)
dataset.head()

,AvgSpeed,CycleVariability,DoubleStance,MeanStepDuration,RoliTronc,Springiness,Sturdiness,Smoothness,Steadiness,Stability,...,critical_displacement_Diffusion_AP_YO,short_time_scaling_Diffusion_AP_YO,long_time_scaling_Diffusion_AP_YO,Frailty_Score,Frailty_State,Fried_Score,Fried_State,FROPCOM0023,SPPB9,SF3603_SCORE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.347786,0.773768,0.250704,7,0,2,0,0.0,17.80,30.0
1,0.880744,0.197969,0.229752,0.564857,0.096065,0.279714,-2.346519,-0.765797,-0.923574,1.968214,...,0.506507,0.840453,0.208258,4,1,3,1,1.0,18.75,22.0
2,1.247801,0.196011,0.254313,0.500417,0.387144,0.706152,-1.213314,-0.587174,0.820538,-0.728560,...,0.161995,0.842991,0.065370,9,0,2,0,0.0,18.86,29.0
3,0.942554,0.233782,0.208736,0.585556,0.100685,0.080144,-1.633020,-0.009619,0.594280,1.739947,...,1.660041,0.871164,0.168832,7,0,2,0,0.0,16.88,29.0
4,1.123115,0.229497,0.257278,0.563333,0.143611,0.375284,-0.206021,-0.733068,-0.058489,-0.218821,...,4.860132,0.838219,-0.295727,8,0,1,0,0.0,12.86,29.0


In [4]:
dataset.shape

(97, 173)

In [5]:
# Models
models = {
    'DecisionTree': DecisionTreeClassifier(class_weight='balanced'),
    'XGBoost': XGBClassifier(scale_pos_weight=2.7),
    'RandomForest': RandomForestClassifier(class_weight='balanced'),
}

# Hyperparameters
params = {
    'DecisionTree': {
        'max_depth': [3, 6, 10],
        'max_features': ['sqrt', 'log2'],
        'criterion': ['gini', 'entropy'],
        #'min_samples_leaf': [10],
},        



    'XGBoost': {
        'n_estimators': [50, 100],
        'max_depth': [3, 7, 10],
        'learning_rate': [0.1, 0.2],
        'min_child_weight': [2, 4],
        },

    'RandomForest': {
        'n_estimators': [50, 100],
        'max_depth': [5, 10],
        'max_features': ['sqrt', 'log2'],
        'criterion': ['gini', 'entropy'],
        'bootstrap': [True, False],
        #'max_leaf_nodes': [3, 6, 9]
    }
}

In [6]:
n_splits = 5

feature_names = dataset.drop(['Fried_State',
                              'Fried_Score',
                              'Frailty_State','Frailty_Score'], axis=1).columns

# Features
X = dataset.drop(['Fried_State',
                  'Fried_Score',
                  'Frailty_State', 'Frailty_Score'], axis=1).values

# Target variable
y = dataset['Frailty_State'].values

model_metrics = {}
tpr_dict = {}
feature_importances = {model_name: [] for model_name in models.keys()}


skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Function to calculate running average and standard deviation
def calculate_running_stats(metric_list):
    n = len(metric_list)
    avg = sum(metric_list) / n
    std = statistics.stdev(metric_list) if n > 1 else 0.0
    return avg, std

for train_index, test_index in skf.split(X, y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  # Impute missing values on train set
  X_train_imputed, best_k = processing(X_train, n_splits=n_splits, k_values=[1, 3, 5, 7, 9], verbose=False)

  X_test_imputed, best_k = processing(X_test, n_splits=n_splits, k_values=[1, 3, 5, 7, 9], verbose=False)

  # Loop through each model and perform grid
  for model_name, model in models.items():
    print(f'Performing Grid Search for {model_name}...')

    # Initialize RFECV
    selector = RFECV(estimator=model, step=1, cv=5, scoring='roc_auc')

    # Fit RFECV
    selector = selector.fit(X_train_imputed, y_train)

    # Transform the datasets to only include the selected features
    X_train_selected = selector.transform(X_train_imputed)

    X_test_selected = selector.transform(X_test_imputed)


    grid_search = GridSearchCV(estimator=model,
                               param_grid=params[model_name],
                               scoring='roc_auc',
                               cv=5,
                               n_jobs=-1,
                               verbose=1)

    # Initialize the metrics for the current model if not already done
    if model_name not in model_metrics:
        model_metrics[model_name] = {
            'f1_score': [],
            'rec_score': [],
            'roc_auc_score': [],
            'best_params' : [],
            'best_models': []
        }

    grid_search.fit(X_train_selected, y_train)

    best_params = grid_search.best_params_
    #print(f'Best Parameters: {best_params}')

    best_score = grid_search.best_score_
    #print(f'Best Score: {best_score}')

    best_model = grid_search.best_estimator_
    #print(f'Best Model: {best_model}')

    y_pred = best_model.predict(X_test_selected)

    y_pred_proba = best_model.predict_proba(X_test_selected)[:, 1]

    # Store the best parameters and model
    model_metrics[model_name]['best_params'].append(best_params)
    model_metrics[model_name]['best_models'].append(best_model)

    model_metrics[model_name]['f1_score'].append(f1_score(y_test, y_pred, average='weighted'))
    model_metrics[model_name]['rec_score'].append(recall_score(y_test, y_pred, average='weighted'))
    model_metrics[model_name]['roc_auc_score'].append(roc_auc_score(y_test, y_pred_proba))

    # Update TPR list for the model
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    if model_name not in tpr_dict:
      tpr_dict[model_name] = []
    tpr_dict[model_name].append(np.interp(np.linspace(0, 1, 100), fpr, tpr))

    # Calculate running average and standard deviation for metrics
    for metric in ['f1_score', 'rec_score', 'roc_auc_score']:
        avg, std = calculate_running_stats(model_metrics[model_name][metric])
        print(f"Running Average {metric}: {avg}, Running Std Dev: {std}")

    # Feature names for selected features
    selected_features = np.array(feature_names)[selector.support_]
    print(f"Selected Features: {selected_features}")


    # If the model has feature importances, store them
    if hasattr(best_model, 'feature_importances_'):
      feature_importances[model_name].append(best_model.feature_importances_)

metrics_to_avg = ['f1_score', 'rec_score', 'roc_auc_score']

# Calculate average and std metrics over all folds
for model_name in models.keys():
    for metric in metrics_to_avg:  # Update this line
        average_metric = sum(model_metrics[model_name][metric]) / n_splits
        std_metric = statistics.stdev(model_metrics[model_name][metric]) if len(model_metrics[model_name][metric]) > 1 else 0.0
        model_metrics[model_name][metric] = {'average': average_metric, 'std': std_metric}


# Display metrics
for model_name, metrics in model_metrics.items():
    print(f"Model: {model_name}")
    for metric, values in metrics.items():
        print(f"Average {metric}: {values['average']}, {values['std']}")
        print("=" * 30)

# Calculate average TPR over all folds
average_tpr_dict = {}
for model_name in models.keys():
    sum_tpr = np.sum(np.array(tpr_dict[model_name]), axis=0)
    average_tpr = sum_tpr / n_splits
    average_tpr[-1] = 1.0  # ensure the curve ends at 1
    average_tpr_dict[model_name] = average_tpr

# Plot ROC Curve
##plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', alpha=0.8)

#for model_name, average_tpr in average_tpr_dict.items():
    #average_fpr = np.linspace(0, 1, 100)  # consistent with your np.linspace in the loop
    #average_auc = np.trapz(average_tpr, average_fpr)  # Trapezoidal rule for AUC
    #plt.plot(average_fpr, average_tpr, label=f"{model_name} (AUC = {average_auc:.2f})")

#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
#plt.title('ROC Curve')
#plt.legend(loc='lower right')
#plt.show()


# Calculate average feature importances and plot them
#for model_name, importances in feature_importances.items():
  #avg_importance = np.mean(importances, axis=0)
  #sorted_idx = np.argsort(avg_importance)[::-1][:20]  # top 20 features

  #plt.figure(figsize=(12, 6))
  #plt.barh(range(20), avg_importance[sorted_idx])
  #plt.yticks(range(20), np.array(dataset.columns)[sorted_idx])
  #plt.xlabel('Average Feature Importance')
  #plt.title(f'Average Feature Importance for {model_name}')
  #plt.show()






Performing Grid Search for DecisionTree...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Running Average f1_score: 0.890625, Running Std Dev: 0.0
Running Average rec_score: 0.9, Running Std Dev: 0.0
Running Average roc_auc_score: 0.8, Running Std Dev: 0.0
Selected Features: ['frequency_quotient_Power_Spectrum_Density_ML_YF'
 'frequency_quotient_Power_Spectrum_Density_AP_YF'
 'short_time_diffusion_Diffusion_ML_YF'
 'long_time_diffusion_Diffusion_ML_YF' 'critical_time_Diffusion_ML_YF'
 'critical_displacement_Diffusion_ML_YF'
 'short_time_scaling_Diffusion_ML_YF' 'long_time_scaling_Diffusion_ML_YF'
 'short_time_diffusion_Diffusion_AP_YF'
 'long_time_diffusion_Diffusion_AP_YF' 'critical_time_Diffusion_AP_YF'
 'critical_displacement_Diffusion_AP_YF'
 'short_time_scaling_Diffusion_AP_YF' 'long_time_scaling_Diffusion_AP_YF'
 'mean_value_ML_YO' 'mean_value_AP_YO' 'mean_distance_Radius_YO'
 'energy_content_above_2_Power_Spectrum_Density_ML_YO'
 'energy_content_above_2_Power_Spect

TypeError: list indices must be integers or slices, not str

In [ ]:
print('Optimal fetures {}'.format(selected_features))
print(len(selected_features))

In [9]:
selector.n_features_

121